In [1]:
from groq import Groq
import os 
import json
import sqlparse
from IPython.display import Markdown
import duckdb
import glob
import yaml

In [ ]:
api_key=os.getenv("GROQ_API_KEY")
groq = Groq(api_key=api_key)

